In [ ]:
!wget https://huggingface.co/datasets/OpenRL/DeepFakeFace/resolve/main/inpainting.zip
!wget https://huggingface.co/datasets/OpenRL/DeepFakeFace/resolve/main/wiki.zip

--2024-05-15 17:53:44--  https://huggingface.co/datasets/OpenRL/DeepFakeFace/resolve/main/inpainting.zip
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/54/4354080778713efff252f32d0ea8cda241f8af84fe016fdd4e0d57db34b94861/795b751f0967b2d940fbd9825e05c541666ec6f8c634acc7640044542897b7b2?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27inpainting.zip%3B+filename%3D%22inpainting.zip%22%3B&response-content-type=application%2Fzip&Expires=1716054824&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjA1NDgyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80My81NC80MzU0MDgwNzc4NzEzZWZmZjI1MmYzMmQwZWE4Y2RhMjQxZjhhZjg0ZmUwMTZmZGQ0ZTBkNTdkYjM0Yjk0ODYxLzc5NWI3NTFmMDk2N2IyZDk0MGZiZDk4MjVlMDVjNTQxNjY2ZWM2

In [ ]:
!unzip -q inpainting.zip
!unzip -q wiki.zip


!rm -r inpainting.zip
!rm -r wiki.zip

In [ ]:
import os
import shutil

def copy_images(source_folder, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Walk through each subfolder in the source folder
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            # Check if the file is an image
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                # Get the full path of the image
                source_path = os.path.join(root, file)
                # Copy the image to the destination folder
                shutil.copy(source_path, destination_folder)

In [ ]:
!rm -r real
!rm -r fake
!rm -r real_vs_fake

rm: cannot remove 'real': No such file or directory
rm: cannot remove 'fake': No such file or directory
rm: cannot remove 'real_vs_fake': No such file or directory


In [ ]:
# Specify the source folder containing subfolders with images
destination_folder = 'real'
# Specify the destination folder where images will be copied
source_folder = 'wiki'

# Call the function to copy images
copy_images(source_folder, destination_folder)

In [ ]:
# Specify the source folder containing subfolders with images
destination_folder = 'fake'
# Specify the destination folder where images will be copied
source_folder = 'inpainting'

# Call the function to copy images
copy_images(source_folder, destination_folder)

In [ ]:
import os
from PIL import Image
import shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm

In [ ]:
def compute_fft_magnitude_spectrum(image):
    """
    Compute the magnitude spectrum of the 2D Fourier Transform of an image.

    Parameters:
    image (PIL.Image.Image): The input image.

    Returns:
    numpy.ndarray: The magnitude spectrum of the image.
    """
    # Convert the image to grayscale
    gray_image = image.convert('L')

    # Compute the 2D Fourier Transform
    fourier_transform1 = np.fft.fft2(gray_image)

    # Shift the zero frequency component to the center
    fourier_transform_shifted = np.fft.fftshift(fourier_transform1)

    # Compute the magnitude spectrum
    magnitude_spectrum1 = 20 * np.log(np.abs(fourier_transform_shifted) + 1e-8)

    return magnitude_spectrum1

def process_images(input_folder, output_folder):
    """
    Process all images in the input folder, compute their magnitude spectrum,
    and save the results in the output folder.

    Parameters:
    input_folder (str): The path to the input folder containing images.
    output_folder (str): The path to the output folder to save the magnitude spectra.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path)

            magnitude_spectrum = compute_fft_magnitude_spectrum(image)

            # Save the magnitude spectrum as an image
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_fourier.png")
            plt.imsave(output_path, magnitude_spectrum, cmap='gray')

In [ ]:
# Process the images in the "real" folder
process_images('real', 'real_fourier')

  0%|          | 64/30000 [00:10<1:25:56,  5.80it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
100%|██████████| 30000/30000 [1:14:57<00:00,  6.67it/s]


In [ ]:
!zip -r real_fourier.zip real_fourier

In [ ]:
# Process the images in the "fake" folder
process_images('fake', 'fake_fourier')

100%|██████████| 30000/30000 [1:28:23<00:00,  5.66it/s]


In [ ]:
!zip -r fake_fourier.zip fake_fourier

In [ ]:
!du -h real_fourier.zip

7.6G	real_fourier.zip


In [ ]:
!du -h fake_fourier.zip

9.1G	fake_fourier.zip


In [ ]:
!cp /content/real_fourier.zip /content/drive/MyDrive/UPC/TER/Fourier/

In [ ]:
!cp /content/fake_fourier.zip /content/drive/MyDrive/UPC/TER/Fourier/

In [ ]:
import os

def count_files_in_folder(folder_path):
    """
    Count the number of files in a folder.

    Parameters:
    folder_path (str): The path to the folder.

    Returns:
    int: The number of files in the folder.
    """
    file_count = 0
    for filename in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, filename)):
            file_count += 1
    return file_count

# Example usage:
real_folder_count = count_files_in_folder('real')
fake_folder_count = count_files_in_folder('fake')

print(f"Number of files in 'real' folder: {real_folder_count}")
print(f"Number of files in 'fake' folder: {fake_folder_count}")

real_folder_count = count_files_in_folder('real_fourier')
fake_folder_count = count_files_in_folder('fake_fourier')

print(f"Number of files in 'real_fourier' folder: {real_folder_count}")
print(f"Number of files in 'fake_fourier' folder: {fake_folder_count}")


# Else

In [ ]:
!mkdir real_vs_fake

In [ ]:
import os
import shutil
import random

# Define paths
source_folder = "real_fourier"
destination_folder = "real_vs_fake"

# Create destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Create test, train, and valid folders within destination folder
folders = ['test', 'train', 'valid']
for folder in folders:
    os.makedirs(os.path.join(destination_folder, folder, 'real'), exist_ok=True)

# List all images in source folder
images = os.listdir(source_folder)
random.shuffle(images)

# Calculate the number of images for each category
total_images = len(images)
test_count = 5000
train_count = int(0.8 * (total_images - test_count))
valid_count = total_images - test_count - train_count

# Move images to respective folders
for folder, count in [('test', test_count), ('train', train_count), ('valid', valid_count)]:
    for i in range(count):
        image = images.pop()
        shutil.copy(os.path.join(source_folder, image), os.path.join(destination_folder, folder, 'real'))

# Copy train, valid, and test folders to real_vs_fake
# for folder in folders:
#     shutil.copytree(os.path.join(destination_folder, folder), os.path.join(destination_folder, 'real_vs_fake', folder))


In [ ]:
import os
import shutil
import random

In [ ]:
# Define paths
source_folder = "fake_fourier"
destination_folder = "real_vs_fake"

# Create destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Create test, train, and valid folders within destination folder
folders = ['test', 'train', 'valid']
for folder in folders:
    os.makedirs(os.path.join(destination_folder, folder, 'fake'), exist_ok=True)

# List all images in source folder
images = os.listdir(source_folder)
random.shuffle(images)

# Calculate the number of images for each category
total_images = len(images)
test_count = 5000
train_count = int(0.8 * (total_images - test_count))
valid_count = total_images - test_count - train_count

# Move images to respective folders
for folder, count in [('test', test_count), ('train', train_count), ('valid', valid_count)]:
    for i in range(count):
        image = images.pop()
        shutil.copy(os.path.join(source_folder, image), os.path.join(destination_folder, folder, 'fake'))

# Copy train, valid, and test folders to real_vs_fake
# for folder in folders:
#     shutil.copytree(os.path.join(destination_folder, folder), os.path.join(destination_folder, 'real_vs_fake', folder))


In [ ]:
# !rm -r /kaggle/working/real_vs_fake
# !rm -r /kaggle/working/fake
# !rm -r /kaggle/working/real

In [ ]:
def count_files(folder_path):
    # Initialize count
    file_count = 0

    # Walk through the folder and count files
    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

In [ ]:
# Specify the folder path
folders = ['test', 'train', 'valid']
folder_path = 'real_vs_fake/'

for i in folders:
  print("Number of files in fake/"+str(i)+" :", count_files(folder_path+i+'/fake'))

In [ ]:
# Specify the folder path
folders = ['test', 'train', 'valid']
folder_path = 'real_vs_fake/'

for i in folders:
  print("Number of files in real/"+str(i)+" :", count_files(folder_path+i+'/real'))

In [ ]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm

In [ ]:
import keras

In [ ]:
base_path = 'real_vs_fake/'
image_gen = ImageDataGenerator(rescale=1./255)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [ ]:
image_gen1 = ImageDataGenerator(rescale=1./255)

valid_flow = image_gen1.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [ ]:
image_gen1 = ImageDataGenerator(rescale=1./255.)
test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
def build_model(pretrained, learning_rate=0.0001):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),
        metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()]
    )

    return model

In [ ]:
densenet = DenseNet121(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5)  # Stop training if no improvement after 3 epochs
checkpoint = ModelCheckpoint('DenseNetImagenetFourier.model.keras', monitor='val_loss', save_best_only=True)  # Save the best model

In [ ]:
# !rm /kaggle/working/DenseNetNoW.keras

In [ ]:
history = model.fit(
    train_flow,
    epochs = 5,
    validation_data =valid_flow,
    callbacks=[checkpoint]
)

In [ ]:
import pickle
with open('trainHistoryDenseNetImagenetFourier.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, mean_absolute_error, mean_squared_error, accuracy_score

# Accuracy and loss history
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, mean_absolute_error, mean_squared_error, accuracy_score

# Accuracy and loss history
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation precision values
plt.figure(figsize=(12, 6))
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation recall values
plt.figure(figsize=(12, 6))
plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('Model recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation AUC values
plt.figure(figsize=(12, 6))
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# from tensorflow.keras.models import load_model
# model2 = build_model(densenet)
# model2.compile()
# # Load the model
# model2.load_weights('/kaggle/working/DenseNetNoW.weights.h5')

In [ ]:
# from tensorflow.keras.models import load_model


# model2 = build_model(densenet)

# # Load the model
# model2.load_weights('DenseNetImagenet.model.keras')

# Valid

In [ ]:
loss, accuracy, auc, precision, recall = model.evaluate(test_flow)

In [ ]:
!cp -r /content/DenseNetImagenetFourier.model.keras /content/drive/MyDrive/UPC/TER